In [1]:
import h5py
import numpy as np
import sys
sys.path.insert(0,"../DeepMIMO/")

from train import nmse, ifft_transform, reconstruction, construction

In [2]:
uplink_path = "../DeepMIMO/DeepMIMO Dataset/dataset_I_2p4_64_ofdm_seperate_bs.npy"
downlink_path = "../DeepMIMO/DeepMIMO Dataset/dataset_I_2p5_64_ofdm_seperate_bs.npy"
locations_path = "../DeepMIMO/DeepMIMO Dataset/location_dataset_I_2p4.npy"
c = 3e8  # in m/s.
Fs = 0.02e9 # in Hz or 0.02 GHz.
Ts = 1/Fs  # sampling time is 1/sampling frequency.
uplink_freq = 2.4e9 # 2.4 GHz
downlink_freq = 2.5e9 # 2.5 GHz
num_bs = 4
bs_index = 0 # first basestation that is 3

In [3]:
[bs_index+num_bs*i for i in range(32//4)]

[0, 4, 8, 12, 16, 20, 24, 28]

In [4]:
all_uplink_data = np.load(uplink_path)
all_downlink_data = np.load(downlink_path)
all_location_data = np.load(locations_path)
num_rows, _= all_uplink_data.shape

# take all the data for basestation with index bs_index
bs_data_index = [bs_index + num_bs*i for i in range(num_rows//num_bs)] # stores the required indices

uplink_data = all_uplink_data[bs_data_index]
downlink_data = all_downlink_data[bs_data_index]
location_data = all_location_data[bs_data_index]

# converting to time domain
uplink = ifft_transform(reconstruction(uplink_data))
downlink = ifft_transform(reconstruction(downlink_data))
location_distances = np.linalg.norm(location_data, axis=1)
num_users, num_samples = uplink.shape
print(uplink.shape, downlink.shape,location_distances.shape)

(151402, 64) (151402, 64) (151402,)


In [5]:
# calculating the multipler in the formulae.
time_index = np.arange(num_samples)
multiplier = (np.zeros(uplink.shape) + np.expand_dims(location_distances, axis=1)/c) + \
                np.expand_dims(time_index*Ts, axis=0)
multipler = np.exp(-(1j*2*np.pi*(downlink_freq - uplink_freq)*multiplier))
print(multiplier.shape)

(151402, 64)


In [6]:
dowlink_prediction = uplink*multipler
print(dowlink_prediction.shape)

(151402, 64)


In [7]:
print("The nmse of the benchmark is",nmse(downlink, dowlink_prediction))

The nmse of the benchmark is 1.3448066592138297
